# Предобработка каротажей

В рамках шага предобработки для каждой скважины производится:
* отбор каротажей, определенных на предыдущем шаге для каждой скважины
* слияние нескольких каротажей одного типа в один
* построение маски коллектор/не коллектор

In [1]:
import os
import warnings

import numpy as np
import pandas as pd

Определение пути к файлу с результатами интерпретации и его загрузка:

In [2]:
INTERPRETATION_PATH = "path/to/interpretation"
interpretation_df = pd.read_csv(INTERPRETATION_PATH, delimiter=";", encoding="cp1251")

Определение пути к директории с каротажами в формате `.npy`:

In [3]:
WELL_NPY_PATH = "path/to/npy"

Определение пути к директории с результатами предобработки каротажей в формате `.npz`:

In [4]:
WELL_NPZ_PATH = "path/to/npz"

if not os.path.exists(WELL_NPZ_PATH):
    os.makedirs(WELL_NPZ_PATH, exist_ok=True)

Загрузка наименований каротажей, измеренных для большинства скважин месторождения, полученных на предыдущем шаге:

In [5]:
COMMON_MNEMONICS_PATH = "path/to/common/mnemonics"
common_mnemonics = np.load(COMMON_MNEMONICS_PATH)

Загрузка названий скважин, для каждой из которых были сняты все каротажи из выбранного списка:

In [6]:
WELL_LIST_PATH = "path/to/wells"
well_names = np.load(WELL_LIST_PATH)

В случае, если для одной скважины было произведено несколько измерений одного и того же типа каротажа, производится их объединение в один. Каждый элемент результирующего картажа — это первый непустой элемент в последовательности значений входных каротажей на той же глубине, упорядоченной по убыванию числа непустых элементов.

In [7]:
def merge_logs(logs):
    priorities = np.sum(~np.isnan(logs), axis=1)
    priorities = np.argsort(priorities, kind="stable")[::-1]
    merged_curve = np.full(logs.shape[1], np.nan)
    for pr in priorities:
        merged_curve = np.where(np.isnan(merged_curve), logs[pr], merged_curve)
    return merged_curve

def select_logs(data, meta, mnemonic):
    logs = data[np.where(meta == mnemonic)[0]]
    if logs.shape[0] == 1:
        return logs[0]
    return merge_logs(logs)

Построение бинарной маски коллектора на основе данных РИГИС:

In [8]:
def find_nearest_index(array, value):
    return (np.abs(np.asarray(array) - value)).argmin()

def generate_mask(dept, well_name, interpretation_df):
    collector_df = interpretation_df[(interpretation_df["Номер скважины"] == well_name) &
                                 (interpretation_df["Признак коллектора"] == 1)]
    top = collector_df["Кровля пропластка"]
    bot = collector_df["Подошва пропластка"]

    mask = np.zeros_like(dept)
    for t_val, b_val in zip(top, bot):
        t_ix = find_nearest_index(dept, t_val)
        b_ix = find_nearest_index(dept, b_val)
        mask[t_ix:b_ix] = 1

    return mask

Запуск процесса предобработки:

In [9]:
%%time

STEP = 0.1

for well_name in well_names:
    data = np.load(os.path.join(WELL_NPY_PATH, well_name + "_data.npy"))
    meta = np.load(os.path.join(WELL_NPY_PATH, well_name + "_meta.npy"))

    dept = select_logs(data, meta, "DEPT")
    if not np.allclose(dept[1:] - dept[:-1], STEP):
        warnings.warn(f"The logs of well {well_name} have a DEPT step, different from {STEP}")
    logs = np.array([select_logs(data, meta, mnemonic) for mnemonic in common_mnemonics])
    mask = generate_mask(dept, well_name, interpretation_df)

    logs_path = os.path.join(WELL_NPZ_PATH, well_name)
    np.savez(logs_path, dept=dept, logs=logs, mask=mask, mnemonics=common_mnemonics)

CPU times: user 7.76 s, sys: 1.65 s, total: 9.41 s
Wall time: 14.8 s
